# Difüzyon Temelli Metin Üretimi (SE Data Set ile)
Bu notebook, HuggingFace'den alınan `salihturkoglu/se_data_set` veri setinin `instruction` ve `response` sütunlarını kullanarak difüzyon temelli metin üretimini gösterir.

## 1. Veri Setini İndir ve Hazırla
Veri seti HuggingFace Datasets ile yüklenir. Her bir örnek için `instruction` giriş, `response` ise hedef metindir.

In [1]:
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# HuggingFace veri setini yükle
dataset = load_dataset('salihturkoglu/se_data_set', split='train')

# Tüm örnekleri kullan (877 satır var)
instructions = [ex['instruction'] for ex in dataset]
responses = [ex['response'] for ex in dataset]

## 2. Tokenizer ve Sözlük Oluşturma
Tüm metinlerden bir kelime sözlüğü oluşturulur ve metinler tokenlara çevrilir.

In [2]:
from collections import Counter

def tokenize(text):
    return text.lower().strip().split()

# Sözlük oluştur
PAD_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'
all_texts = instructions + responses
counter = Counter()
for text in all_texts:
    counter.update(tokenize(text))

vocab_list = [PAD_TOKEN, UNK_TOKEN] + [tok for tok, count in counter.items() if count >= 2][:10000]
vocab = {tok: idx for idx, tok in enumerate(vocab_list)}
reverse_vocab = {idx: tok for tok, idx in vocab.items()}

def encode(text):
    return [vocab.get(tok, vocab[UNK_TOKEN]) for tok in tokenize(text)]

def decode(token_ids):
    return ' '.join([reverse_vocab.get(idx, UNK_TOKEN) for idx in token_ids if idx != vocab[PAD_TOKEN]])

## 3. PyTorch Dataset ve DataLoader
Instruction ve response çiftlerini uygun şekilde tensor haline getirir.

In [3]:
class InstructionResponseDataset(Dataset):
    def __init__(self, instructions, responses, vocab, max_len=64):
        self.inputs = [encode(text)[:max_len] for text in instructions]
        self.targets = [encode(text)[:max_len] for text in responses]
        self.max_len = max_len
        self.vocab = vocab

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        inp = self.inputs[idx]
        tgt = self.targets[idx]
        inp = inp + [self.vocab['<PAD>']] * (self.max_len - len(inp))
        tgt = tgt + [self.vocab['<PAD>']] * (self.max_len - len(tgt))
        return torch.tensor(inp, dtype=torch.long), torch.tensor(tgt, dtype=torch.long)

max_len = 128
dataset = InstructionResponseDataset(instructions, responses, vocab, max_len=max_len)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## 4. Difüzyon Süreci: Gürültü Ekleme ve Çıkarma

In [4]:
def add_noise(batch, noise_level=0.5):
    noisy = batch.clone()
    mask = (torch.rand(noisy.shape) < noise_level)
    random_tokens = torch.randint(2, len(vocab), noisy.shape, device=batch.device)
    noisy[mask] = random_tokens[mask]
    return noisy

## 5. Model Tanımı

In [5]:
class DiffusionTextModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=2048, num_layers=8, nhead=8):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim, nhead=nhead, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x, src_key_padding_mask=None):
        emb = self.embedding(x)
        out = self.transformer(emb, src_key_padding_mask=src_key_padding_mask)
        out = self.fc(out)
        return out

## 6. Eğitim Süreci

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DiffusionTextModel(len(vocab)).to(device)

def train_diffusion_model(model, dataloader, epochs=15, noise_level=0.5):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss(ignore_index=vocab['<PAD>'])
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_inputs, batch_targets in dataloader:
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            noisy_inputs = add_noise(batch_targets, noise_level)
            mask = (batch_targets == vocab['<PAD>'])
            optimizer.zero_grad()
            outputs = model(noisy_inputs, src_key_padding_mask=mask)
            loss = criterion(outputs.view(-1, outputs.size(-1)), batch_targets.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

train_diffusion_model(model, dataloader, epochs=15, noise_level=0.5)

Epoch 1, Loss: 7.2675
Epoch 2, Loss: 6.5326
Epoch 3, Loss: 5.9995
Epoch 4, Loss: 5.6588
Epoch 5, Loss: 5.4110
Epoch 6, Loss: 5.2345
Epoch 7, Loss: 5.0627
Epoch 8, Loss: 4.9098
Epoch 9, Loss: 4.7601
Epoch 10, Loss: 4.6605
Epoch 11, Loss: 4.5139
Epoch 12, Loss: 4.4244
Epoch 13, Loss: 4.3298
Epoch 14, Loss: 4.2664
Epoch 15, Loss: 4.1858


## 7. Metin Üretimi (Difüzyon ile Response Oluşturma)

In [7]:
def generate_response(model, instruction, steps=8, max_len=32):
    model.eval()
    inp = encode(instruction)[:max_len]
    inp = inp + [vocab['<PAD>']] * (max_len - len(inp))
    inp_tensor = torch.tensor([inp], dtype=torch.long, device=device)
    # Başlangıçta tamamen rastgele bir dizi
    generated = torch.randint(2, len(vocab), (1, max_len), device=device)
    for step in range(steps):
        mask = (generated == vocab['<PAD>'])
        with torch.no_grad():
            outputs = model(generated, src_key_padding_mask=mask)
            predicted = outputs.argmax(dim=-1)
        prob = 1.0 - (step + 1) / steps
        random_mask = (torch.rand_like(generated.float()) < prob)
        generated[random_mask] = predicted[random_mask]
    tokens = generated[0].tolist()
    return decode(tokens)

# Örnek bir instruction ile response üret
test_instruction = instructions[0]
print('Instruction:', test_instruction)
print('Gerçek Response:', responses[0])
print('Model Response:', generate_response(model, test_instruction))

Instruction: Yazılım Mühendisliği bölümünün ders programını görebilir miyim?
Gerçek Response: Güncel ders programını bölüm web sitesindeki duyurular bölümündeki en güncel ders programı duyurusuna ulaşarak görebilirsiniz.
Model Response: öğrenci ve eğitimi ve fazla i̇şyeri i̇şyeri öğrenci eğitimi i̇şyeri bilgi i̇şyeri ise, ile ders ile ders en ve en eğitimi ile ders bölümün ile eğitimi öğrenci ders için i̇şyeri ve eğitimi


c:\Users\emreq\Desktop\Transformers\.venv\Lib\site-packages\torch\nn\modules\transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\NestedTensorImpl.cpp:182.)
  output = torch._nested_tensor_from_mask(


## 9. Test: Herhangi Bir Soru ile Modeli Deneyin
Aşağıdaki hücrede, istediğiniz bir soruyu `test_instruction` değişkenine yazarak modelin cevabını görebilirsiniz.

In [8]:
# Test etmek istediğiniz soruyu buradan değiştirebilirsiniz.
test_instruction = "Ders kaydı yaparken üst sınıftan ders alabilir miyim?"

print('Instruction:', test_instruction)
print('Gerçek Response:', responses[instructions.index(test_instruction)] if test_instruction in instructions else "Yok")
print('Model Response:', generate_response(model, test_instruction, max_len=max_len))

Instruction: Ders kaydı yaparken üst sınıftan ders alabilir miyim?
Gerçek Response: Yok
Model Response: akademik veya yapılır. öğrenci öğrenci ve ders ders ve ders olan üzerinden ve öğrenci akademik veya öğrenci teknoloji ve veya yönetimi için ders öğrenci veya ve ve bölümün bilgi öğrenci veya ders en olan resmi öğrenci akademik akademik akademik sistemi öğrenci en öğrenci ders öğrenci ile bir öğrenci öğrenci ve bir istediği öğrenci ve ve genellikle sayfasından tüm sistemi öğrenci olmak ilgili değişim öğrenci ders ders öğrenci ve sistemi onaylandıktan öğrenci ders genellikle akademik ve veya ders ders üniversite öğrenci hizmetlerinden fakültesi ve içindeki ve ile bilgi öğrenci ders ve olanakları bir öğrenci akademik veya ve ders bilgi ders öğrenci (katkı ve 4. için ve için da bilgi istediği bilgiye ve fazla ve bir sağlamak ve akademik olan ve ve akademik ve ve sağlanır. öğrenci bir web öğrenci


## 10. Modelin Test Edilmesi
Aşağıdaki hücrede modelin test verisi üzerinde ne kadar doğru response üretebildiği ölçülür. Basit bir doğruluk metriği olarak, modelin response üretiminde orijinal response ile token bazında ne kadar örtüştüğü hesaplanır.

In [9]:
def evaluate_diffusion_model(model, dataset, n_samples=100, steps=8):
    model.eval()
    total = 0
    correct = 0
    for i in range(min(n_samples, len(dataset))):
        inp, tgt = dataset[i]
        inp = inp.unsqueeze(0).to(device)
        tgt = tgt.unsqueeze(0).to(device)
        generated = torch.randint(2, len(vocab), tgt.shape, device=device)
        for step in range(steps):
            mask = (generated == vocab['<PAD>'])
            with torch.no_grad():
                outputs = model(generated, src_key_padding_mask=mask)
                predicted = outputs.argmax(dim=-1)
            prob = 1.0 - (step + 1) / steps
            random_mask = (torch.rand_like(generated.float()) < prob)
            generated[random_mask] = predicted[random_mask]
        mask = (tgt != vocab['<PAD>'])
        total += mask.sum().item()
        correct += ((generated == tgt) & mask).sum().item()
    accuracy = correct / total if total > 0 else 0.0
    print(f"Test doğruluğu: {accuracy:.2%} ({correct}/{total})")

# Test et
evaluate_diffusion_model(model, dataset, n_samples=100, steps=8)

Test doğruluğu: 1.71% (60/3509)
